In [22]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime

from sklearn.metrics import mean_squared_error

In [2]:
imb = pd.read_csv("../data/imbalance_1h.csv")

In [3]:
imb.head()

,datetime,1,2,weekday,date
0,2013-12-31 23:00:00,917.118,92.620,1,2013-12-31
1,2014-01-01 00:00:00,1264.266,82.465,2,2014-01-01
2,2014-01-01 01:00:00,746.810,50.305,2,2014-01-01
3,2014-01-01 02:00:00,-15.512,-42.080,2,2014-01-01
4,2014-01-01 03:00:00,-36.904,-28.995,2,2014-01-01


In [4]:
imb.drop(['2', 'weekday', 'date'], axis=1, inplace=True)

In [6]:
imb = imb.set_index('datetime')

In [8]:
imb.shape

(64776, 1)

In [9]:
imb['1h'] = imb['1'].shift(1)

In [10]:
imb.head()

,1,1h
datetime,,
2013-12-31 23:00:00,917.118,NaN
2014-01-01 00:00:00,1264.266,917.118
2014-01-01 01:00:00,746.810,1264.266
2014-01-01 02:00:00,-15.512,746.810
2014-01-01 03:00:00,-36.904,-15.512


In [11]:
imb.tail()

,1,1h
datetime,,
2021-05-22 18:00:00,-1.051667,-16.731500
2021-05-22 19:00:00,14.628167,-1.051667
2021-05-22 20:00:00,30.308000,14.628167
2021-05-22 21:00:00,164.239000,30.308000
2021-05-22 22:00:00,298.170000,164.239000


In [19]:
imb = imb.iloc[1:]

In [21]:
# train test split
train = imb.iloc[:int(len(imb)*0.7)]
test = imb.iloc[int(len(imb)*0.7):]

In [23]:
rmse = np.sqrt(mean_squared_error(test['1'], test['1h']))
print(rmse)

464.102293740116
